In [1]:
#export
import sys
from os.path import join
import math

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))
from operations import *

In [2]:
#export
def init_weight_he(in_dim, num_hidden):
    return torch.randn(in_dim, num_hidden) * (2./in_dim) ** 0.5

def init_weight_norm(in_dim, num_hidden):
    return torch.randn(in_dim, num_hidden) * (2./in_dim) ** 0.5

def init_weight(in_dim, num_hidden, end=False):
    if end:
        return init_weight_norm(in_dim, num_hidden)
    else:
        return init_weight_he(in_dim, num_hidden)

def init_bias_zero(num_hidden):
    return torch.zeros(num_hidden)

def init_bias_uni(num_hidden):
    return torch.randn(num_hidden)

def init_bias(num_hidden, zero=True):
    if zero:
        return init_bias_zero(num_hidden)
    else:
        return init_bias_uni(num_hidden)

In [3]:
#export
class Parameter():
    # own data struct instead of acyclic directed graph
    def __init__(self, data, requires_grad=True):
        self.data = data
        self.requires_grad = requires_grad
        self.grad = 0.
        
    def __get__(self, instance, owner):
        return self.data
    
    def __repr__(self):
        return f'shape: {tuple(self.data.shape)}, grad: {self.requires_grad}'
    
    def step(self, learning_rate):
        self.data -= learning_rate * self.grad
    
    def zero_data(self):
        self.data.zero_()
        
    def zero_grad(self):
        self.grad = 0.
    
    def update(self, grad):
        self.grad = grad

In [4]:
#export
class Sequential():
    def __init__(self, *args):
        self.layers = list(args)
        self.training = True
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def __repr__(self):
        return '(Sequential)\n\t' + '\n\t'.join(f'(Layer{i}) {layer}' for i, layer in enumerate(self.layers, 1))
    
    def train(self):
        self.training = True
    
    def eval_(self):
        self.training = False
        
    def backward(self):
        for layer in reversed(self.layers):
            layer.backward()
        
    def parameters(self):
        for layer in self.layers:
            for parameter in layer.parameters():
                yield parameter

In [5]:
#export
class Module():
    def __init__(self):
        self._parameters = {}
        
    def __setattr__(self, k, v):
        if isinstance(v, Parameter): 
            self._parameters[k] = v
        super().__setattr__(k, v)
        
    def __call__(self, *args):
        self.args = args
        self.out = self.fwd(*args)
        return self.out
    
    def parameters(self):
        for p in self._parameters.values():
            yield p
    
    def forward(self):
        raise NotImplementedError('Module.forward')
    
    def backward(self):
        self.bwd(self.out, *self.args)

In [6]:
#export
def softmax(inp):
    # prone to overflow (floating aint precise)
    return inp.exp() / inp.exp().sum(-1, keepdim=True)

def log_sum_exp(inp):
    e = inp.max(-1)[0]
    return e + (inp - e[:, None]).exp().sum(-1).log()
    
def log_softmax(inp):
    # LogSumExp trick to avoid floating point error
    return inp - log_sum_exp(inp).unsqueeze(-1)

def nll_loss(pre, tar):
    # use multiple indexing 
    return -pre[range(tar.shape[0]), tar].mean()

def cross_entropy(inp, tar):
    return nll_loss(log_softmax(inp), tar)

In [7]:
#export
class Linear(Module):
    def __init__(self, in_dim, num_hidden, end=False, require_grad=True):
        super().__init__()
        self.w = Parameter(init_weight(in_dim, num_hidden, end), require_grad)
        self.b = Parameter(init_bias(num_hidden), require_grad)
    
    def __repr__(self):
        return f'Linear({self.w.data.shape[0]}, {self.w.data.shape[1]})'
        
    def fwd(self, inp):
        return inp @ self.w.data + self.b.data
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.data.t()
        self.w.update(inp.t() @ out.g)
        self.b.update(out.g.sum(0))

class ReLU(Module):
    def __repr__(self):
        return f'ReLU()'
    
    def fwd(self, inp):
        return inp.clamp_min(0.) - 0.5
    
    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g
        
class CrossEntropy(Module):
    def __repr__(self):
        return f'(CrossEntropy)'
        
    def fwd(self, inp, tar):
        return cross_entropy(inp, tar)
    
    def bwd(self, loss, inp, tar):
        inp_soft = softmax(inp)
        inp_soft[range(tar.shape[0]), tar] -= 1
        inp.g = inp_soft / tar.shape[0]

In [8]:
nh = 50

x_train, y_train, x_valid, y_valid = get_mnist_data()
(num_data, in_dim), out_dim = x_train.shape, int(y_train.max() + 1)

In [9]:
model = Sequential(Linear(in_dim, nh), 
                   ReLU(), 
                   Linear(nh, out_dim, True))
loss_fn = CrossEntropy()

In [10]:
model

(Sequential)
	(Layer1) Linear(784, 50)
	(Layer2) ReLU()
	(Layer3) Linear(50, 10)

In [11]:
for p in model.parameters():
    print(p)

shape: (784, 50), grad: True
shape: (50,), grad: True
shape: (50, 10), grad: True
shape: (10,), grad: True


In [12]:
loss = loss_fn(model(x_train), y_train)
loss_fn.backward()
model.backward()

xtg = x_train.g.clone()
w1g = model.layers[0].w.grad.clone()
b1g = model.layers[0].b.grad.clone()
w2g = model.layers[2].w.grad.clone()
b2g = model.layers[2].b.grad.clone()

x_train2 = x_train.clone().requires_grad_(True)
model.layers[0].w.data.requires_grad_(True)
model.layers[0].b.data.requires_grad_(True)
model.layers[2].w.data.requires_grad_(True)
model.layers[2].b.data.requires_grad_(True)

loss = loss_fn(model(x_train2), y_train)
loss.backward()

test_near(w1g, model.layers[0].w.data.grad)
test_near(b1g, model.layers[0].b.data.grad)
test_near(w2g, model.layers[2].w.data.grad)
test_near(b2g, model.layers[2].b.data.grad)